In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
import warnings
import openai
from dotenv import load_dotenv
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [2]:
load_dotenv()
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY2"))

In [3]:
df = pd.read_csv('../cleaning_before_eval/ready_to_eval.csv')
df.head()

,No,Question,Dr Answer,Claude Answer,Qwen Answer,GPT Answer,Deepseek RAG Answer,Deepseek non RAG Answer,Claude Full Answer,Qwen Full Answer,GPT Full Answer,Deepseek RAG Full Answer,Deepseek non RAG Full Answer
0,1.0,Pasien mengalami demam pada waktu malam. Sebel...,"Demam Tifoid 70%, Demam Berdarah Dengue 30%","Demam Tifoid, Leptospirosis, Demam Berdarah De...","Demam Biasa, Dispepsia, Asma, Infeksi Saluran ...","Demam Biasa, Demam Tifoid, Demam Berdarah Deng...","Demam Tifoid (Typhoid Fever), Malaria, Leptosp...","Malaria, Infeksi Saluran Kemih (ISK) atau Piel...","Berdasarkan gejala yang disampaikan, saya akan...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan gejala yang dialami pasien, yaitu ...",Berdasarkan gejala yang Anda sebutkan—demam pa...
1,2.0,Pasien mengalami buang air besar cair lebih da...,"Gastroenteritis (GE) Akut 70%, Intoleransi Lak...","Gastroenteritis (GE) Akut, Disentri, Infeksi E...","Diare Akut, Gastroenteritis, Dispepsia, Infeks...","Gastroenteritis Akut, Infeksi Escherichia coli...","Gastroenteritis Akut, Keracunan Makanan, Infek...","Gastroenteritis Akut, Keracunan Makanan, Koler...","Berdasarkan informasi klinis yang diberikan, s...","Berdasarkan gejala yang disampaikan, yaitu bua...","Berdasarkan informasi yang diberikan, pasien m...","Berdasarkan informasi yang diberikan, pasien m...",Berdasarkan gejala yang Anda deskripsikan—diar...
2,3.0,Pasien datang dengan keluhan demam selama 6 ha...,DBD 80%,"Demam Tifoid, Malaria, DBD, Leptospirosis, Inf...","Demam Biasa, Dispepsia, Asma, Demam Dengue, Lu...","Demam Berdarah Dengue, Demam Tifoid, Malaria, ...","Demam Berdarah Dengue (DBD), Demam Tifoid, Lep...","Demam Berdarah Dengue (DBD) / Dengue Fever, Ch...","Berdasarkan informasi yang diberikan, saya aka...","Berdasarkan gejala yang disampaikan, yaitu dem...",Berdasarkan keluhan yang disampaikan oleh pasi...,Berdasarkan keluhan pasien yang meliputi demam...,Berdasarkan gejala yang Anda sebutkan—demam ≥6...
3,4.0,Pasien menderita demam disertai munculnya brun...,"Infeksi Bakteri/Virus (Sistemik) 50%, Infeksi ...","DBD, Infeksi Bakteri/Virus (Sistemik), Infeksi...","Demam Biasa, Dispepsia, Asma, Infeksi Lokal, A...","Infeksi Kulit (Dermatitis atau Selulitis), Kej...","Infeksi Bakteri (Selulitis), Reaksi Alergi (De...",Erisipelas (Infeksi Bakteri pada Kulit dan Jar...,"Berdasarkan informasi klinis yang diberikan, s...","Berdasarkan gejala yang disampaikan, yaitu dem...","Berdasarkan informasi yang diberikan, pasien m...",### Analisis Gejala dan Kemungkinan Diagnosis\...,Berdasarkan gejala dan temuan pemeriksaan yang...
4,5.0,Pasien nyeri dada sejak 4 jam lalu seperti ter...,"Angina Pekrotis 80%, Infark Miokard Akut (Sera...","Angina Pekrotis, Infark Miokard Akut (Serangan...","Infark Miokard Akut (Serangan Jantung), Dispep...","Angina Pekrotis, Infark Miokard, Penyakit Musk...","Angina Pectoris (Kardiovaskular), Infark Mioka...","Infark Miokard Akut (Serangan Jantung), Angina...",Berdasarkan informasi yang diberikan dalam kon...,"Berdasarkan gejala yang disampaikan, yaitu nye...","Berdasarkan informasi yang diberikan, pasien m...",### Analisis Gejala\r\n\r\nPasien mengeluhkan ...,Berdasarkan gejala nyeri dada seperti terhimpi...


In [4]:
with open('kamus.json', 'r') as file:
    SYNONYM_MAP = json.load(file)
    
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    for abbr, canonical in SYNONYM_MAP.items():
        abb = abbr.lower()
        canon = canonical.lower()

        if (text == canon): break
        pattern = rf'\b{re.escape(abb)}\b'
        if re.search(pattern, text):
            text = re.sub(pattern, canon, text)
            break
        
    return text

In [5]:
import time

def llm_judge_answer(question, candidate_answer:str, gt_answers:str, model_name="gpt-3.5-turbo", max_retries=3):
    for attempt in range(max_retries):
        try:
            if attempt > 0:
                delay = 2 ** attempt  
                print(f"Retrying after {delay} seconds...")
                time.sleep(delay)
            
            normalized_candidate = normalize_text(candidate_answer)
            ground_truth_text = normalize_text(gt_answers)

            prompt = f"""
            You are a medical expert evaluating diagnosis answers. Please judge if the candidate answer is appropriate given the ground truth.

            Question: {question}
            Ground Truth Answers: {ground_truth_text}
            Candidate Answer: {normalized_candidate}

            Please evaluate if the candidate answer is:
            1. Correct and matches the ground truth
            2. Partially correct or related
            3. Incorrect or unrelated

            Provide your response as a JSON object with the following structure:
            {{
                "score": <score from 0 to 1, where 1 is perfect match between ground truth and candidate answer>,
                "reasoning": "<brief explanation of your judgment>",
                "category": "<correct|partial|incorrect>"
            }}

            Response:"""

            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a medical expert evaluating diagnosis accuracy."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=200
            )

            response_text = response.choices[0].message.content
            # print(response_text)
            try:
                start_idx = response_text.find('{')
                end_idx = response_text.rfind('}') + 1
                json_str = response_text[start_idx:end_idx]
                
                result = json.loads(json_str)
                if 'score' in result and 'reasoning' in result and 'category' in result:
                    return result
                else:
                    raise ValueError("Invalid response structure")
                    
            except Exception as parse_error:
                print(f"Parse error on attempt {attempt + 1}: {parse_error}")
                if attempt == max_retries - 1:
                    return {
                        "score": 0.5,
                        "reasoning": "Could not parse LLM response after multiple attempts",
                        "category": "partial"
                    }
                continue
                
        except Exception as e:
            print(f"API error on attempt {attempt + 1}: {e}")
            if "insufficient_quota" in str(e) or "quota" in str(e).lower():
                print("⚠️  OpenAI API quota exceeded. Please check your billing.")
                return {
                    "score": 0.0,
                    "reasoning": "API quota exceeded",
                    "category": "error"
                }
            elif attempt == max_retries - 1:
                return {
                    "score": 0.0,
                    "reasoning": f"API error after {max_retries} attempts: {str(e)}",
                    "category": "error"
                }
            continue

    return {
        "score": 0.0,
        "reasoning": "Unknown error occurred",
        "category": "error"
    }

In [6]:
sample_question = "Pasien mengalami demam pada waktu malam"
sample_candidate = "Demam biasa"
sample_gt = "Demam Tifoid 70%, Demam Berdarah Dengue 30%"

test_result = llm_judge_answer(sample_question, sample_candidate, sample_gt)
print(f"Test Result 3.5: {test_result}")
test_result = llm_judge_answer(sample_question, sample_candidate, sample_gt, model_name='gpt-4-0613')
print(f"Test Result 4: {test_result}")


Test Result 3.5: {'score': 0.0, 'reasoning': "The candidate answer 'common fever' does not match any of the specific diagnoses mentioned in the ground truth (demam tifoid, demam berdarah, demam berdarah dengue).", 'category': 'incorrect'}
Test Result 4: {'score': 0.2, 'reasoning': "The candidate answer 'common fever' is a very general term and does not specify the type of fever, while the ground truth answers are specific types of fevers (typhoid fever, dengue fever). Therefore, the candidate answer is partially correct but lacks specificity.", 'category': 'partial'}


In [7]:
def find_score_llm_judge(candidate_answers, question, gt_answers, model_name):
    if not candidate_answers or not gt_answers:
        return None, 0.0, {}
    
    best_score = 0.0
    average_score = 0.0
    
    for candidate in tqdm(candidate_answers, desc=f'Processing cands {model_name}'):
        judgment = llm_judge_answer(question, candidate, gt_answers, model_name)
        score = judgment.get('score', 0.0)
        average_score += score
        
        if score > best_score:
            best_score = score
    
    cands = ",".join(candidate_answers)
    all_round_score = llm_judge_answer(question, cands, gt_answers, model_name).get('score', 0.0)
    
    return (average_score/len(candidate_answers)), best_score, all_round_score

In [8]:
def process_row(row):
    print(f"Processing row {row.name}...")
    # truth_list = [item.strip() for item in row['Dr Answer'].split(',')]
    model_cols = {
        'Claude Answer': 'LLM_JUDGE_Claude',
        'Qwen Answer': 'LLM_JUDGE_Qwen',
        'GPT Answer': 'LLM_JUDGE_GPT',
        'Deepseek RAG Answer': 'LLM_JUDGE_Deepseek_RAG',
        'Deepseek non RAG Answer': 'LLM_JUDGE_Deepseek_nonRAG'
    }
    # gpt-3.5-turbo
    # 'gpt-4-0613'
    for col_name, result_col in tqdm(model_cols.items(), desc='Processing models'):
        model_list = [item.strip() for item in row[col_name].split(',')]
        row[f'{result_col}_avg_4'], row[f'{result_col}_best_4'], \
            row[f'{result_col}_all_4'] = find_score_llm_judge(model_list, row['Question'], row['Dr Answer'], model_name='gpt-4-0613')
        row[f'{result_col}_avg_3.5'], row[f'{result_col}_best_3.5'], \
            row[f'{result_col}_all_3.5'] = find_score_llm_judge(model_list, row['Question'], row['Dr Answer'], model_name='gpt-3.5-turbo')
    
    return row

In [9]:
df2 = df.head(10)
df2 = df2.apply(process_row, axis=1)

Processing row 0...


Processing models: 100%|██████████| 5/5 [02:28<00:00, 29.73s/it]


Processing row 1...


Processing models: 100%|██████████| 5/5 [02:30<00:00, 30.08s/it]


Processing row 2...


...

Processing models: 100%|██████████| 5/5 [02:20<00:00, 28.16s/it]


Processing row 8...


Processing models: 100%|██████████| 5/5 [02:13<00:00, 26.74s/it]


Processing row 9...


Processing models: 100%|██████████| 5/5 [02:10<00:00, 26.20s/it]


In [14]:
df3 = df.iloc[10:20]
df3 = df3.apply(process_row, axis=1)

Processing row 10...


Processing models: 100%|██████████| 5/5 [02:11<00:00, 26.20s/it]


Processing row 11...


Processing models: 100%|██████████| 5/5 [02:18<00:00, 27.66s/it]


Processing row 12...


...

Processing models: 100%|██████████| 5/5 [02:11<00:00, 26.29s/it]


Processing row 18...


Processing models: 100%|██████████| 5/5 [01:56<00:00, 23.38s/it]


Processing row 19...


Processing models: 100%|██████████| 5/5 [02:04<00:00, 24.97s/it]


In [9]:
df4 = df.iloc[20:30]
df4 = df4.apply(process_row, axis=1)

Processing row 20...


Processing models: 100%|██████████| 5/5 [02:39<00:00, 31.90s/it]


Processing row 21...


Processing models: 100%|██████████| 5/5 [02:21<00:00, 28.37s/it]


Processing row 22...


...

Processing models: 100%|██████████| 5/5 [02:38<00:00, 31.70s/it]


Processing row 28...


Processing models: 100%|██████████| 5/5 [01:35<00:00, 19.20s/it]


Processing row 29...


Processing models: 100%|██████████| 5/5 [02:07<00:00, 25.57s/it]


In [12]:
df5 = df.iloc[30:]
df5 = df5.apply(process_row, axis=1)

Processing row 30...


Processing models: 100%|██████████| 5/5 [02:51<00:00, 34.22s/it]


Processing row 31...


Processing models: 100%|██████████| 5/5 [02:20<00:00, 28.01s/it]


Processing row 32...


...

Processing models: 100%|██████████| 5/5 [02:33<00:00, 30.68s/it]


Processing row 43...


Processing models: 100%|██████████| 5/5 [02:26<00:00, 29.29s/it]


Processing row 44...


Processing models: 100%|██████████| 5/5 [02:02<00:00, 24.50s/it]


In [ ]:
# ================= LLM-Based Judge Evaluation =================
print("================ LLM-Based Judge Evaluation ================")

# LLMs to evaluate (excluding ground truth)
llms_to_evaluate = [
    "Qwen 2.5 72B",
    "GPT-4o mini", 
    "Deepseek-V3 (non RAG)",
    "Deepseek-V3-RAG"
]

# Store all results
all_results_llm_judge = {}

for llm_name in llms_to_evaluate:
    print(f"\n{'='*50}")
    print(f"Evaluating {llm_name} (LLM-based judge)")
    print(f"{'='*50}")
    
    try:
        llm_grouped = group_llm_answers(llm_name)
        results = evaluate_llm_grouped_judge(llm_grouped, gt_grouped)
        all_results_llm_judge[llm_name] = results
        
        # Calculate summary statistics
        best_scores = [r['best_score'] for r in results]
        matches = sum(r['match'] for r in results)
        
        print(f"\nSummary for {llm_name} (LLM-based judge):")
        print(f"Matches: {matches} / {len(results)}")
        print(f"Best Score - Mean: {np.mean(best_scores):.3f}, Std: {np.std(best_scores):.3f}")
        
    except Exception as e:
        print(f"Error evaluating {llm_name}: {e}")
        all_results_llm_judge[llm_name] = []

print(f"\nLLM-based judge evaluation completed for all LLMs!")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create detailed results dataframe for LLM-based judge
detailed_results_llm = []
for llm_name, results in all_results_llm_judge.items():
    if results:  # Only process if results exist
        for result in results:
            detailed_results_llm.append({
                'LLM': llm_name,
                'No': result['No'],
                'Best_Score': result['best_score'],
                'Match': result['match'],
                'Category': result['judgment'].get('category', 'unknown'),
                'Reasoning': result['judgment'].get('reasoning', 'No reasoning provided')
            })

if detailed_results_llm:
    detailed_df_llm = pd.DataFrame(detailed_results_llm)
    detailed_df_llm.to_csv('llm_diagnosis_evaluation_results_llm_judge.csv', index=False)
    print("Detailed LLM-based judge results saved to 'llm_diagnosis_evaluation_results_llm_judge.csv'")
    
    # Summary statistics for LLM-based judge
    summary_data_llm = []
    for llm_name, results in all_results_llm_judge.items():
        if results:  # Only process if results exist
            best_scores = [r['best_score'] for r in results]
            matches = [r['match'] for r in results]
            summary_data_llm.append({
                'LLM': llm_name,
                'Best_Score_Mean': np.mean(best_scores),
                'Best_Score_Std': np.std(best_scores),
                'Match_Rate': np.mean(matches),
                'Best_Scores': best_scores,
                'Matches': matches
            })
    
    if summary_data_llm:
        summary_df_llm = pd.DataFrame(summary_data_llm)
        print("\nSummary Statistics (LLM-based judge):")
        print(summary_df_llm[['LLM', 'Best_Score_Mean', 'Match_Rate']].round(3))
        
        # Display sample of detailed results
        print("\nSample of detailed results:")
        print(detailed_df_llm.head(10))
    else:
        print("No summary data to display - all evaluations may have failed")
else:
    print("No detailed results to save - all evaluations may have failed")


In [ ]:
# Visualize LLM-based judge results
if 'summary_df_llm' in locals() and not summary_df_llm.empty:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. Bar plot of mean best scores
    axes[0].bar(summary_df_llm['LLM'], summary_df_llm['Best_Score_Mean'], alpha=0.7, color='blue')
    axes[0].set_title('Mean Best Score (LLM Judge)')
    axes[0].set_ylabel('Score')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].set_ylim(0, 1)
    
    # 2. Box plot of best scores
    all_scores_llm = []
    all_labels_llm = []
    for _, row in summary_df_llm.iterrows():
        all_scores_llm.extend(row['Best_Scores'])
        all_labels_llm.extend([row['LLM']] * len(row['Best_Scores']))
    
    if all_scores_llm:
        score_df_llm = pd.DataFrame({'LLM': all_labels_llm, 'Best_Score': all_scores_llm})
        sns.boxplot(data=score_df_llm, x='LLM', y='Best_Score', ax=axes[1])
        axes[1].set_title('Best Score Distribution by LLM (LLM Judge)')
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].set_ylim(0, 1)
    
    # 3. Bar plot of match rate
    axes[2].bar(summary_df_llm['LLM'], summary_df_llm['Match_Rate'], alpha=0.7, color='green')
    axes[2].set_title('Match Rate by LLM (LLM Judge)')
    axes[2].set_ylabel('Match Rate')
    axes[2].set_ylim(0, 1)
    axes[2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Overall LLM Ranking (by mean best score)
    print("\nOverall LLM Ranking (by Mean Best Score):")
    ranking = summary_df_llm.sort_values('Best_Score_Mean', ascending=False)
    for i, (_, row) in enumerate(ranking.iterrows()):
        print(f"{i+1}. {row['LLM']}: {row['Best_Score_Mean']:.3f}")
    
    # Overall LLM Ranking (by Match Rate)
    print("\nOverall LLM Ranking (by Match Rate):")
    ranking_match = summary_df_llm.sort_values('Match_Rate', ascending=False)
    for i, (_, row) in enumerate(ranking_match.iterrows()):
        print(f"{i+1}. {row['LLM']}: {row['Match_Rate']:.3f}")
        
else:
    print("No visualization data available - evaluations may have failed")


In [ ]:
# Performance Analysis by Question (LLM Judge)
if 'detailed_df_llm' in locals() and not detailed_df_llm.empty:
    print("Performance Analysis by Question (LLM Judge):")
    print("=" * 50)
    
    # Group by question number to see which questions are harder
    question_performance = detailed_df_llm.groupby('No').agg({
        'Best_Score': ['mean', 'std'],
        'Match': ['mean']
    }).round(3)
    
    question_performance.columns = ['Best_Score_Mean', 'Best_Score_Std', 'Match_Rate']
    question_performance = question_performance.reset_index()
    
    print("Questions with lowest average scores:")
    worst_questions = question_performance.nsmallest(5, 'Best_Score_Mean')
    print(worst_questions)
    
    print("\nQuestions with highest average scores:")
    best_questions = question_performance.nlargest(5, 'Best_Score_Mean')
    print(best_questions)
    
    # Category analysis
    print("\nCategory Distribution:")
    category_counts = detailed_df_llm['Category'].value_counts()
    print(category_counts)
    
else:
    print("No performance data available for analysis")


In [ ]:
# EVALUATION SUMMARY (LLM Judge)
print("LLM-BASED JUDGE EVALUATION SUMMARY")
print("=" * 50)

if 'summary_df_llm' in locals() and not summary_df_llm.empty:
    # Best performing LLMs
    best_score_idx = summary_df_llm['Best_Score_Mean'].idxmax()
    best_match_idx = summary_df_llm['Match_Rate'].idxmax()
    
    best_semantic = summary_df_llm.loc[best_score_idx]
    best_match = summary_df_llm.loc[best_match_idx]
    
    print(f"Best LLM by Score: {best_semantic['LLM']} (Score: {best_semantic['Best_Score_Mean']:.3f})")
    print(f"Best LLM by Match Rate: {best_match['LLM']} (Rate: {best_match['Match_Rate']:.3f})")
    
    # Overall statistics
    print(f"\nOverall Statistics:")
    print(f"Total Questions Evaluated: {len(gt_grouped)}")
    print(f"Total LLMs Evaluated: {len(summary_df_llm)}")
    print(f"Average Score Across All LLMs: {summary_df_llm['Best_Score_Mean'].mean():.3f}")
    print(f"Average Match Rate Across All LLMs: {summary_df_llm['Match_Rate'].mean():.3f}")
    
    # Recommendations
    print("\nRECOMMENDATIONS:")
    print("1. Use LLM-based judge scores for nuanced medical diagnosis evaluation.")
    print("2. Consider both mean score and match rate for comprehensive assessment.")
    print("3. Review low-scoring questions for common diagnostic challenges.")
    print("4. Use category analysis to understand error types and patterns.")
    
    print("\nEvaluation completed successfully!")
    print("Check 'llm_diagnosis_evaluation_results_llm_judge.csv' for detailed results.")
    
else:
    print("Evaluation failed or no results available.")
    print("Please check your OpenAI API key and internet connection.")
    print("You may need to run the evaluation cells again.")
